In [2]:
# Dependencies and Setup
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states


# Census API Key
c = Census(census_key, year=2017)

In [61]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("B01003_001E", "B19013_001E","B19301_001E","B17001_002E","B23025_005E",), {
                         'county:{}'.format(county_fips)})

NameError: name 'county_fips' is not defined

In [15]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population","Household Income","Per Capita Income","Poverty Count","Poverty Rate","Unemployment Count", "Unemployment Rate"]]

census_pd.head()

,Zipcode,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count,Unemployment Rate
0,00601,17599.0,11757.0,7041.0,11282.0,64.105915,2454.0,13.943974
1,00602,39209.0,16190.0,8978.0,20428.0,52.100283,2538.0,6.473004
2,00603,50135.0,16645.0,10897.0,25176.0,50.216416,3588.0,7.156677
3,00606,6304.0,13387.0,5960.0,4092.0,64.911168,204.0,3.236041
4,00610,27590.0,18741.0,9266.0,12553.0,45.498369,1474.0,5.342515


In [16]:
census_pd['Zipcode'] = census_pd.astype({"Zipcode": str})

In [17]:
census_pd['Zipcode'].head()

0    00601
1    00602
2    00603
3    00606
4    00610
Name: Zipcode, dtype: object

In [5]:
len(census_pd)

33120

In [49]:
# File to Load (Remember to Change These)
zipcodes = "uszips.csv"

# Read Purchasing File and store into Pandas data frame
zipcode_table = pd.read_csv("uszips.csv", encoding="ISO-8859-1",converters={'Zipcode': '{:0>5}'.format,'county_fips':'{:0>5}'.format})

zipcode_table.head()

,Zipcode,state_id,county_fips,county_name
0,00951,PR,72137,Toa Baja
1,96863,HI,15003,Honolulu
2,34141,FL,12021,Collier
3,78060,TX,48297,Live Oak
4,78144,TX,48255,Karnes


In [50]:
zipcode_table['Zipcode'] = zipcode_table.astype({"Zipcode": str})


In [29]:
merged_df = pd.merge(census_pd,zipcode_table,how='left',on='Zipcode')
merged_df = merged_df.dropna(how='any')
merged_df.head()

,Zipcode,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count,Unemployment Rate,COUNTYNAME,STATE,STCOUNTYFP
0,00601,17599.0,11757.0,7041.0,11282.0,64.105915,2454.0,13.943974,Adjuntas Municipio,PR,72001
1,00601,17599.0,11757.0,7041.0,11282.0,64.105915,2454.0,13.943974,Ponce Municipio,PR,72113
2,00602,39209.0,16190.0,8978.0,20428.0,52.100283,2538.0,6.473004,Aguada Municipio,PR,72003
3,00602,39209.0,16190.0,8978.0,20428.0,52.100283,2538.0,6.473004,Aguadilla Municipio,PR,72005
4,00603,50135.0,16645.0,10897.0,25176.0,50.216416,3588.0,7.156677,Aguadilla Municipio,PR,72005


In [54]:
# File to Load
CDC_Data = "CDC_Data.csv"

# Read Purchasing File and store into Pandas data frame
CDC_Table = pd.read_csv("CDC_Data.csv", encoding="ISO-8859-1",converters={'STCOUNTYFP': '{:0>5}'.format})
CDC_Table['STCOUNTYFP'] = CDC_Table.astype({"STCOUNTYFP": str})
CDC_Table.head()

,STCOUNTYFP,County,Deaths,Population,Crude Rate,% of Total Deaths
0,01003,"Baldwin County, AL",19,212628,Unreliable,0.00%
1,01043,"Cullman County, AL",12,82755,Unreliable,0.00%
2,01049,"DeKalb County, AL",11,71617,Unreliable,0.00%
3,01055,"Etowah County, AL",26,102755,25.3,0.10%
4,01073,"Jefferson County, AL",167,659197,25.3,0.40%


In [56]:
new_merged_df = pd.merge(merged_df,CDC_Table,how='outer',on='STCOUNTYFP')
new_merged_df = new_merged_df.dropna(how='any')
new_merged_df.head()

,Zipcode,Population_x,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count,Unemployment Rate,COUNTYNAME,STATE,STCOUNTYFP,County,Deaths,Population_y,Crude Rate,% of Total Deaths
267,01001,17537.0,57694.0,33135.0,1557.0,8.878371,422.0,2.406341,Hampden County,MA,25013,"Hampden County, MA",111,469818,23.6,0.20%
268,01008,1272.0,62750.0,33933.0,89.0,6.996855,59.0,4.638365,Hampden County,MA,25013,"Hampden County, MA",111,469818,23.6,0.20%
269,01009,384.0,59440.0,26043.0,171.0,44.531250,22.0,5.729167,Hampden County,MA,25013,"Hampden County, MA",111,469818,23.6,0.20%
270,01010,3724.0,86523.0,39077.0,156.0,4.189044,164.0,4.403867,Hampden County,MA,25013,"Hampden County, MA",111,469818,23.6,0.20%
271,01011,1507.0,75263.0,30903.0,102.0,6.768414,50.0,3.317850,Hampden County,MA,25013,"Hampden County, MA",111,469818,23.6,0.20%


In [43]:
len(new_merged_df)

501

In [ ]:
plt.scatter(x_axis, data, marker="o", facecolors="red", edgecolors="black",
            s=x_axis, alpha=0.75)

In [ ]:
#chart ideas: heatmap with CDC deaths and poverty rate/unemployment/percapita income
#general trending over 20 years of opioid deaths in USA versus total pop

